In [41]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix



In [42]:
# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


In [43]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = r'C:/Users/honey/Documents/placment work/Electronical-AI-Device-Recogniser/Theo/CSV/acs-f2-datasetOG.csv'
df = pd.read_csv(file_path)

# Separating features and target (assuming 'equipment' is the target)
X = df.drop(columns=['equipment', 'time'])  # Dropping time and target columns
y = df['equipment']  # Assuming equipment is the label

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train.values, y_train.values))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test.values, y_test.values))

# Shuffle and batch the datasets
BATCH_SIZE = 32

train_dataset = train_dataset.shuffle(len(X_train)).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)


In [44]:
# Split features and labels
X = df.drop(columns=['time', 'equipment'])
y = df['equipment']


In [45]:
# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [46]:
# # Convert labels to one-hot encoding
# y_encoded = to_categorical(y)


In [47]:
# Encode categorical labels into integers
le = LabelEncoder()
y_encoded_int = le.fit_transform(y)

# Convert labels to one-hot encoding
y_encoded = to_categorical(y_encoded_int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [48]:
# Define the model with architectural improvements
#changes made:  Increased neurons in first layer# Batch normalization to stabilize learning,Higher dropout for regularization, More neurons for deeper architecture, Another batch normalization layer
# Softmax for multi-class classification, A third hidden layer with 128 neurons
model = Sequential([
    Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),  
    Dropout(0.3),  
    Dense(256, activation='relu'),  
    BatchNormalization(), 
    Dropout(0.3),  # Regularization
    Dense(128, activation='relu'),  
    Dense(y_encoded.shape[1], activation='softmax')  
])

# Learning rate scheduler for adaptive learning rate
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)

c:\Users\honey\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [49]:
# Compile the model with the learning rate schedule
model.compile(optimizer=Adam(learning_rate=lr_schedule),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
# Early stopping callback to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), 
                    epochs=1000, callbacks=[early_stopping])


Epoch 1/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - accuracy: 0.4125 - loss: 1.7003 - val_accuracy: 0.6205 - val_loss: 1.0734
Epoch 2/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.5772 - loss: 1.1674 - val_accuracy: 0.6739 - val_loss: 0.9226
Epoch 3/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6250 - loss: 1.0303 - val_accuracy: 0.6935 - val_loss: 0.8551
Epoch 4/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6457 - loss: 0.9697 - val_accuracy: 0.6922 - val_loss: 0.8468
Epoch 5/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6636 - loss: 0.9260 - val_accuracy: 0.7065 - val_loss: 0.8133
Epoch 6/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6754 - loss: 0.8943 - val_accuracy: 0.7280 - val_loss: 0.7701
Epoch 7/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6780 - loss: 0.8845 - val_accuracy: 0.7334 - val_loss: 0.7449
Epoch 8/1000
4028/4028 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.6884 -

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test loss: {test_loss:.4f}")

In [ ]:
# Evaluate the model performance
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")

In [ ]:
# Generate a classification report
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)
print(classification_report(y_true, y_pred))

In [ ]:
# Make predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()

In [37]:
# Function to predict appliance and probabilities
def predict_appliance(electrical_readings):
    # Ensure the input is a 2D array
    if electrical_readings.ndim == 1:
        electrical_readings = electrical_readings.reshape(1, -1)
    
    # Scale the input
    scaled_input = scaler.transform(electrical_readings)
    
    # Make prediction
    probabilities = model.predict(scaled_input)[0]
    predicted_class = le.inverse_transform([np.argmax(probabilities)])[0]
    
    return predicted_class, probabilities

In [ ]:
# Example usage
example_reading = np.array([50.0, 340, 111.284, -40.89, 0.533, 224.038])
predicted_appliance, probabilities = predict_appliance(example_reading)

print(f"Predicted appliance: {predicted_appliance}")
print("Probabilities for each appliance:")
for appliance, prob in zip(le.classes_, probabilities):
    print(f"{appliance}: {prob:.4f}")


In [ ]:

# Plot probabilities
plt.figure(figsize=(10, 6))
sns.barplot(x=le.classes_, y=probabilities)
plt.title('Appliance Prediction Probabilities')
plt.xlabel('Appliance')
plt.ylabel('Probability')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [40]:
import joblib
joblib.dump(scaler, 'scaler1.0.9.pkl')

model.save('appliance_recogniser#1.0.9.keras')